# Diabetes prevention program model

In [ ]:
from pathlib import Path

import numpy as np
import pandas as pd
import geopandas as gpd

from epxexec import fred_job
from epxexec.fred_job import FREDJob

from diabetes_prevention import (
    VariableMeta,
    get_state_block_groups_gdf,
    get_results_df,
    plot_cume_diagnoses,
)

In [ ]:
DATA_DIR = Path("data")

## 0. Data required

1. Insurance category probabilities by census tract, age, sex, and race provided by PolicyMap
2. Program engagement by insurance category from Alva et al. 2022 
3. Program eligibility from Flegal et al. 2012 dataset used for include_external_data.ipynb. Note that from https://www.cdc.gov/diabetes/prevention/pdf/dprp-standards.pdf, people over the age of 18 are eligible if they have a BMI >= 25 and don’t already have a type-2 diabetes diagnosis
4. Probability of developing diabetes over time from https://www.ncbi.nlm.nih.gov/pmc/articles/PMC3135022/

## 1. Insurace category probabilities

`insurance_status.csv`
Formatted data structure:

- census_tract, 0
- prob_no_insurance, 1
- prob_medicaid_insurance, 2
- prob_medicare_insurance, 3
- prob_private_insurance, 4


Base data extracted from the census by Brian at PolicyMap. Represents total probabilities of being in each category for each census tract.

In [ ]:
col_types = {
    "tract_fips": str,
    "time_frame": str,
    "PPOPWOINS": float,
    "PPOPWMEDICAID": float,
    "PPOPWMEDICARE": float,
    "PPOPWINSPRIV": float,
}
raw_ins_prob_df = pd.read_csv(
    DATA_DIR / "raw/nevada_tracts_insurance_indicators.csv",
    dtype=col_types,
    usecols=col_types.keys(),
)
raw_ins_prob_df.head()

[US Census website](https://www.census.gov/geographies/mapping-files/time-series/geo/cartographic-boundary.html)

In [ ]:
nv_census_tracts_url = (
    "https://www2.census.gov/geo/tiger/GENZ2022/shp/cb_2022_32_tract_500k.zip"
)
nv_census_tracts_gdf = gpd.read_file(nv_census_tracts_url)
nv_census_tracts_gdf.head(2)

Check completeness of census tracts in `raw_ins_prob_df` compared to census geo data

In [ ]:
assert (
    len(
        (
            nv_census_tracts_gdf[["NAMELSAD", "GEOID"]]
            .merge(raw_ins_prob_df, how="left", left_on="GEOID", right_on="tract_fips")
            .pipe(lambda df: df[df["tract_fips"].isna()])
        ).index
    )
    == 0
)

List tracts with missing data in `raw_ins_prob_df`

In [ ]:
missing_data_tracts: pd.Series = raw_ins_prob_df.pipe(
    lambda df: df[df["PPOPWOINS"] == -9999]
)["tract_fips"]

In [ ]:
def find_neighbors(tract_id: str, tracts_gdf: gpd.GeoDataFrame) -> pd.Series:
    tgt_geom = tracts_gdf.pipe(lambda df: df[df["GEOID"] == tract_id]).iloc[0][
        "geometry"
    ]
    neighbor_ids = tracts_gdf.pipe(
        lambda df: df[df["geometry"].touches(tgt_geom)]["GEOID"]
    )
    return neighbor_ids

In [ ]:
def neighbor_closest_to_mean_of_all_neighbors(df):
    var_names = ["PPOPWOINS", "PPOPWINSPRIV", "PPOPWMEDICARE", "PPOPWMEDICAID"]
    mean_point = df[var_names].mean()

    def distance_to_mean(row: pd.Series, mean_point: pd.Series) -> float:
        return np.sqrt(np.sum(np.power((row - mean_point).values, 2)))

    return (
        df.assign(
            dist=lambda df: (
                df[var_names].apply(distance_to_mean, mean_point=mean_point, axis=1)
            )
        )
        .sort_values(by="dist")
        .iloc[0]["neighbor_fips"]
    )

In [ ]:
missing_tracts_replacement_data_df = (
    missing_data_tracts.rename("missing_tract_fips")
    .to_frame()
    .groupby("missing_tract_fips")
    .apply(
        lambda df: (
            find_neighbors(
                df.iloc[0]["missing_tract_fips"], nv_census_tracts_gdf
            ).rename("neighbor_fips")
        )
    )
    .droplevel(1)
    .to_frame()
    .reset_index()
    .merge(raw_ins_prob_df, how="left", left_on="neighbor_fips", right_on="tract_fips")
    .drop(columns=["tract_fips", "time_frame"])
    # remove neighbors that also have missing data
    .pipe(lambda df: df[df["PPOPWOINS"] != -9999])
    # find census tract ID of neighbor most similar to mean of all neighbors
    .groupby("missing_tract_fips")
    .apply(neighbor_closest_to_mean_of_all_neighbors)
    .rename("neighbor_fips")
    .to_frame()
    .reset_index()
    .merge(raw_ins_prob_df, how="left", left_on="neighbor_fips", right_on="tract_fips")
    .drop(columns=["neighbor_fips", "tract_fips"])
    .rename(columns={"missing_tract_fips": "tract_fips"})
)

In [ ]:
missing_tracts_replacement_data_df

In [ ]:
def renormalize(row):
    for v in var_names:
        row[v] = row[v] * row["scale_fac"] / 100
    return row

In [ ]:
var_names = ["PPOPWOINS", "PPOPWINSPRIV", "PPOPWMEDICARE", "PPOPWMEDICAID"]
col_names = {
    "tract_fips": "census_tract",
    "PPOPWOINS": "prob_no_insurance",
    "PPOPWMEDICAID": "prob_medicaid_insurance",
    "PPOPWMEDICARE": "prob_medicare_insurance",
    "PPOPWINSPRIV": "prob_private_insurance",
}
proc_ins_prob_df = (
    pd.concat(
        [
            raw_ins_prob_df.pipe(lambda df: df[df["PPOPWOINS"] != -9999]),
            missing_tracts_replacement_data_df,
        ]
    )
    # Renormalize
    .assign(scale_fac=lambda df: 100 / df[var_names].sum(1))
    .apply(renormalize, axis=1)
    .drop(columns="scale_fac")
    .rename(columns=col_names)
    .loc[:, col_names.values()]
)

In [ ]:
proc_ins_prob_df.to_csv(DATA_DIR / "insurance_status.csv", index=False)

## 2. Prob participate by insurance status

`prop_participate_by_insurance_status.csv`

Structure:
- insurance_status_id  # 0 = no insurance, 1 = medicaid, 2 = medicare, 3 = private
- probability participate

In [ ]:
prob_participate_df = (
    pd.DataFrame.from_records(
        [
            {"status": "private", "sex": "male", "pct": 3.2},
            {"status": "medicare", "sex": "male", "pct": 2.7},
            {"status": "medicaid", "sex": "male", "pct": 1.9},
            {"status": "no_insurance", "sex": "male", "pct": 1.5},
            {"status": "private", "sex": "female", "pct": 3.3},
            {"status": "medicare", "sex": "female", "pct": 3.4},
            {"status": "medicaid", "sex": "female", "pct": 2.8},
            {"status": "no_insurance", "sex": "female", "pct": 0.9},
        ]
    )
    .assign(prob=lambda df: df["pct"] / 100)
    .groupby("status")["prob"]
    .mean()
    .to_frame()
    .join(
        pd.Series(
            {"private": 3, "medicare": 2, "medicaid": 1, "no_insurance": 0}
        ).rename("insurance_status_id")
    )
    .sort_values("insurance_status_id")
    .loc[:, ["insurance_status_id", "prob"]]
)

In [ ]:
prob_participate_df.to_csv(
    DATA_DIR / "prob_participate_by_insurance_status.csv", index=False
)

## 3. Probability overweight

The probability of being overweight differentiated by sex, race, and age group can be obtained from Quickstart Guide Chapter 12. Run through the code in that tutorial, and copy the file `prob_overweight_obese.csv` to the `data` directory in this project.

## 4. Probability develop diabetes over time

Average annual rate obtained from [Diabetes Prevention Program Research Group 2009](https://www.ncbi.nlm.nih.gov/pmc/articles/PMC3135022) Figure 4

In [ ]:
rates_df = (
    pd.Series({"placebo": 10.9, "lifestyle": 5.0})
    .rename("rate_per_hundred_person_years")
    .to_frame()
    .assign(rate_per_year=lambda df: df["rate_per_hundred_person_years"] / 100)
    .assign(
        prob_per_year=lambda df: (
            df.apply(
                lambda row: row["rate_per_year"] * np.exp(-row["rate_per_year"]), axis=1
            )
        )
    )
    .rename_axis("intervention")
    .reset_index()
    .rename_axis("intervention_id")
    .reset_index()
)
rates_df

In [ ]:
rates_df.to_csv(DATA_DIR / "incidence_per_person_per_year.csv", index=False)

## Geographical data

Download US County geographic boundary data in shapefile format from the [US Census website](https://www.census.gov/geographies/mapping-files/time-series/geo/cartographic-boundary.html) and read into a GeoDataFrame.

In [ ]:
nv_block_groups_gdf = get_state_block_groups_gdf("32")

## Model runs

Set `program_available = 1` in `model/main.fred`

In [ ]:
from epxexec.fred_job import FREDJob

In [ ]:
with_program_job = FREDJob(job_id=226)

In [ ]:
with_program_job = fred_job("model/main.fred")

Set `program_available = 0` in `model/main.fred`

In [ ]:
without_program_job = fred_job("model/main.fred")

In [ ]:
without_program_job = FREDJob(job_id=227)

In [ ]:
variables = (
    VariableMeta("n_diagnosed", "diabetes_incidence.csv"),
    VariableMeta("n_participating", "program_participation.csv"),
)

In [ ]:
def average_over_runs(df: pd.DataFrame) -> pd.DataFrame:
    index_cols = ["block_group", "calendar_year", "variable"]
    var_cols = ["value"]
    return df.groupby(index_cols)[var_cols].mean().reset_index()

In [ ]:
with_program_df = get_results_df(
    with_program_job, variables, nv_block_groups_gdf["block_group"], (2023, 2028)
).pipe(average_over_runs)

without_program_df = get_results_df(
    without_program_job, variables, nv_block_groups_gdf["block_group"], (2023, 2028)
).pipe(average_over_runs)

In [ ]:
with_program_df.set_index(['block_group', 'calendar_year', 'variable'])['value'].unstack().reset_index()

In [ ]:
inc_df = (
    pd.concat(
        [
            with_program_df.assign(scenario="with_program"),
            without_program_df.assign(scenario="without_program"),
        ]
    )
    .set_index(["block_group", "calendar_year", "scenario"])
    .unstack("scenario")
)
inc_df.columns = ["_".join(col).strip() for col in inc_df.columns.values]
inc_df = inc_df.assign(
    diff=lambda df: df["n_diagnosed_without_program"] - df["n_diagnosed_with_program"]
)

In [ ]:
with_program_df.pipe(
    lambda df: df[(df["calendar_year"] == 2028) & (df["cume_n_diagnosed"] > 0)]
)["cume_n_diagnosed"].max()

In [ ]:
without_program_df.pipe(
    lambda df: df[(df["calendar_year"] == 2028) & (df["cume_n_diagnosed"] > 0)]
)["cume_n_diagnosed"].max()

## Plot outputs

In [ ]:
results_df = pd.read_csv("results/nv_results.csv")
results_df.head()

In [ ]:
nv_center = {"lat": 38.9, "lon": -116.4}

In [ ]:
plot_cume_diagnoses(
    results_df.pipe(lambda df: df[df["scenario"] == "without_program"]),
    nv_block_groups_gdf,
    "Nevada",
    nv_center,
)

In [ ]:
plot_cume_diagnoses(
    results_df.pipe(lambda df: df[df["scenario"] == "with_program"]),
    nv_block_groups_gdf,
    "Nevada",
    nv_center,
)